## 1 Image-based Style Transfer

Loading the dataset of choice.

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import tensorflow as tf
import keras

# List all physical GPU devices
gpus = tf.config.list_physical_devices('GPU')

In [2]:
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.19.0


In [4]:
print("CUDA Available:", tf.config.list_physical_devices('GPU'))
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

CUDA Available: []
Num GPUs Available:  0


In [5]:
cpu_devices = tf.config.list_physical_devices('CPU')
print("Available CPUs:", cpu_devices)

Available CPUs: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [6]:
GPU_in_use: int = 0
CPU_in_use: int = 0

In [7]:
from helper_functions.device_helper import get_device

In [8]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(gpus[GPU_in_use].name)
else:
    print("No GPU found")

No GPU found


In [ ]:
import tensorflow as tf
import keras

base_image_path = "../demo_images/san.png"
style_reference_image_paths = ["../demo_images/starry_night.png"]
style_reference_path = style_reference_image_paths[0]

In [10]:
original_width, original_height = keras.utils.load_img(base_image_path).size
img_height = 400
img_width = round(original_width * img_height / original_height) 

In [ ]:
from helper_functions.helper import  preprocess_image, deprocess_image
from helper_functions.loss_functions import style_loss, content_loss, total_variation_loss

In [ ]:
total_variation_weight = 1e-6
single_style_weight = 1e-6
single_content_weight = 2.5e-8

In [ ]:
from shared_utils.gatys_network import get_content_layer_names,get_style_layer_names,get_style_weights, get_content_weights

In [ ]:
chosen_loss_network : str = "vgg19"

In [ ]:
style_layer_names = [
 "block1_conv1",
 "block2_conv1",
 "block3_conv1",
 "block4_conv1",
 "block5_conv1",
]
content_layer_names = ["block5_conv2"]
style_weights = {'block1_conv1': 1.,
                 'block2_conv1': 0.8,
                 'block3_conv1': 0.5,
                 'block4_conv1': 0.3,
                 'block5_conv1': 0.1}
content_weights = {'block5_conv2': 1e-6}
use_custom : bool = True
if use_custom:
    style_layer_names = get_style_layer_names(chosen_loss_network)
    content_layer_names = get_content_layer_names(chosen_loss_network)
    style_weights = get_style_weights(chosen_loss_network)
    content_weights = get_content_weights(chosen_loss_network)

In [ ]:
from shared_utils.helper import create_dir

In [ ]:
from shared_utils.network import get_model_for_loss_net

In [ ]:
def get_model(model_name : str = "vgg19",img_width : int = 224,img_height : int = 224,use_model_layers = True):
  """ Creates our model with access to intermediate layers. 
  
  This function will load the VGG19 model and access the intermediate layers. 
  These layers will then be used to create a new model that will take input image
  and return the outputs from these intermediate layers from the VGG model. 
  
  Returns:
    returns a keras model that takes image inputs and outputs the style and 
      content intermediate layers. 
  """
  # Load our model. We load pretrained VGG, trained on imagenet data (weights=’imagenet’)
  vgg = get_model_for_loss_net(model_name,image_size=(img_width,img_height))
  vgg.trainable = False
  
  # Get output layers corresponding to style and content layers 
  if use_model_layers:
     model_outputs = dict([(layer.name, layer.output) for layer in vgg.layers])
  else:
    style_outputs = [vgg.get_layer(name).output for name in style_layer_names]
    content_outputs = [vgg.get_layer(name).output for name in content_layer_names]
    model_outputs = style_outputs + content_outputs
  # Build model 

  return keras.Model(vgg.input, model_outputs)

In [ ]:
feature_extractor = get_model(chosen_loss_network,img_width,img_height)

In [ ]:
from helper_functions.loss_functions import ssim_loss, psnr_loss

In [ ]:
from typing import Optional
import lpips
def compute_custom_losses(combination_image, base_image,custom_losses : bool = True, loss_net = "alex", includes : list[str] = ["ssim", "psnr", "lpips"],weights : dict = {}):
    if custom_losses:    
        loss = 0.0
        if "ssim" in includes:
            ssim_weight = weights.get("ssim", 1.0)
            ssim_loss_value = ssim_loss(combination_image, base_image)
            loss += ssim_loss_value * ssim_weight
        if "psnr" in includes:
            psnr_weight = weights.get("psnr", 1.0)
            psnr_loss_value = psnr_loss(combination_image, base_image)
            loss += psnr_loss_value * psnr_weight 
        if "lpips" in includes:
            lpips_weight = weights.get("lpips", 1.0)
            lpips_loss_fn = lpips.LPIPS(net=loss_net)
            lpips_loss = lpips_loss_fn(base_image, combination_image)
            loss += lpips_loss * lpips_weight
        return loss
    return 0.0

In [ ]:

def compute_loss(combination_image, base_image, style_reference_image,use_l2=False):
  input_tensor = tf.concat(
  [base_image, style_reference_image, combination_image], axis=0)
  features = feature_extractor(input_tensor)
  loss = tf.zeros(shape=())

  content_weight_per_layer : float = single_content_weight / len(content_layer_names)
  for layer_name in content_layer_names:
    layer_features = features[layer_name]
    base_image_features = layer_features[0, :, :, :]
    combination_features = layer_features[2, :, :, :]
    loss += content_weight_per_layer * content_loss(
        base_image_features, combination_features
    )
  style_weight_per_layer : float = single_style_weight / len(style_layer_names)
  for layer_name in style_layer_names:
    layer_features = features[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    style_loss_value = style_loss(
    style_reference_features, combination_features, img_width, img_height)
    loss += style_weight_per_layer * style_loss_value

  custom_loss_weights = {
      "ssim": 1.0,
      "psnr": 1.0,
      "lpips": 1.0,}
  loss += compute_custom_losses(combination_image, base_image,weights=custom_loss_weights)


  loss += total_variation_weight * total_variation_loss(combination_image,use_l2=use_l2)
  return loss

In [ ]:
from tensorflow.keras import layers

Set the policy.

In [ ]:
from tensorflow.keras.mixed_precision import set_global_policy
def control_policy(enable_mixed_precision: bool = False):
    if enable_mixed_precision:
        print("Enabled mixed_float16 policy")
        set_global_policy('mixed_float16')

Call the policy function.

In [ ]:
control_policy(enable_mixed_precision=False)

In [ ]:
def normalization_grads(grads, strength= None) -> list:
    norm = tf.linalg.global_norm(grads)
    if strength:
        norm_grads : list = [g * (strength / (norm + 1e-8)) for g in grads]
    else:
        norm_grads : list = [g / (norm + 1e-8) for g in grads]
    return norm_grads

In [ ]:
import tensorflow as tf

@tf.function
def compute_loss_and_grads(combination_image, base_image, style_images,apply_normalization=False,strength=None):
    tf.print("combination_image == tf.Variable", isinstance(combination_image, tf.Variable))
    type_style_images = style_images if isinstance(style_images,list) else [style_images]
    with get_device(GPU_in_use, CPU_in_use):  
        with tf.GradientTape() as tape:
            loss = tf.zeros(shape=())
            num : int = len(type_style_images)
            style_cal = single_style_weight / num
            # iterate through the style images
            for image in type_style_images:
                style_loss_value = compute_loss(
                    combination_image, base_image, image
                )
                loss += style_cal * style_loss_value

        tf.print("Loss:", loss)
        grads = tape.gradient(loss, combination_image)
        tf.print("Gradient:", grads)
        if apply_normalization:
            grads = normalization_grads(grads)
        return loss, grads

In [ ]:
def preprocess_style_image(style_reference_image_paths):
    images = []
    for path in style_reference_image_paths:
        img = preprocess_image(path,img_width, img_height)
        images.append(img)
    return tf.concat(images, axis=0)
  

In [ ]:
def add_noise_to_image(image,noise_strength : float =0.1):
    noise = tf.random.normal(shape=tf.shape(image), mean=0.0, stddev=noise_strength, dtype=image.dtype)
    noisy_image = image + noise
    return tf.clip_by_value(noisy_image, 0.0, 255.0)

In [ ]:
def preprocess_NST_images(base_image_path : str, style_reference_image_path : str):
    with get_device(GPU_in_use, CPU_in_use):
        base_image = preprocess_image(base_image_path,img_width,img_height)
        style_reference_images = preprocess_image(style_reference_image_path, img_width, img_height)
        initial_combination_image = add_noise_to_image(base_image)
        combination_image = tf.Variable(initial_combination_image)
    return base_image, style_reference_images, combination_image


In [ ]:
import math
import os
import time
import numpy as np

In [ ]:
from datetime import datetime

In [ ]:
from shared_utils.optimizer import get_optimizer

In [ ]:
from helper_functions.checkPointManager import checkPointManager

In [ ]:
def clip_0_1(image, min : float = 0.0, max : float = 1.0):
  return tf.clip_by_value(image, clip_value_min=min, clip_value_max=max)

In [ ]:
@tf.function
def apply_style_transfer_step(combination_image, base_image, style_image, optimizer, clip_image : bool = False):

    with get_device(GPU_in_use, CPU_in_use):
        loss, grads = compute_loss_and_grads(
            combination_image, base_image, style_image
        )
    optimizer.apply_gradients([(grads, combination_image)])
    if clip_image:
        combination_image.assign(clip_0_1(combination_image))
    return loss, grads

In [ ]:
from helper_functions.HardwareLogger import HardwareLogger
from helper_functions.training_helper import result_save
def training_loop(base_image, style_image, combination_image,content_name : str,style_name: str,verbose : int = 0,include_checkpoints : bool = False,config : dict={}):
    chosen_optimizer = config.get("optimizer", 0.01)
    lr = config.get("lr", 0.01)
    w,h = config.get("size",(400,400))
    if isinstance(chosen_optimizer, str):
        optimizer = get_optimizer(chosen_optimizer, learning_rate=lr)
    else:
        print("Invalid passed in optimizer type. Should be a string or a list of strings.\n")
        return
    hardware_logger  = HardwareLogger()
    generated_images = []
    start_step : int = 1
    iterations = 1000
    save_step: int = 100
    best_cost = math.inf
    best_image = None
    if start_step > iterations:
        print(f"Start step ({start_step}) is greater than the specified iterations ({iterations}). No training will be performed.")
        log_data = hardware_logger.get_log()
        return generated_images, best_image, best_cost, hardware_logger
    for i in range(start_step, iterations + 1):
        loss, grads = apply_style_transfer_step(combination_image, base_image, style_image, optimizer)
        if i % save_step == 0:
            # hardware usage
            hardware_logger.log_loss(loss,i)
            img = deprocess_image(combination_image.numpy(), w, h)
            hardware_logger.log_hardware()
            hardware_logger.log_end_check()
            if loss < best_cost:
                best_cost = loss
                best_image = img
            generated_images.append(img)
            result_save(content_name, style_name, i, img)
    hardware_logger.on_training_end()
    log_data = hardware_logger.get_log()
    return generated_images, best_image, best_cost,log_data

In [ ]:
content_folder = "content"
style_folder = "style"

In [ ]:
def get_image_files(folder_path : str,image_file_types=('.png', '.jpg', '.jpeg')):
    return [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.lower().endswith(image_file_types)]

In [ ]:
content_images = get_image_files(content_folder)[0:1]
style_images = get_image_files(style_folder)[0:1]

NameError: name 'content_folder' is not defined

In [ ]:
content_images

In [ ]:
style_images

In [ ]:

def update_model(loss_network: str, img_width: int, img_height: int):
    feature_extractor = get_model(loss_network, img_width, img_height)
    style_layer_names = get_style_layer_names(loss_network)
    content_layer_names = get_content_layer_names(loss_network)
    style_weights = get_style_weights(loss_network)
    content_weights = get_content_weights(loss_network)
    return feature_extractor, style_layer_names, content_layer_names, style_weights, content_weights

In [ ]:
config = {
    "optimizer": "adam",
    "lr": 0.01,
    "threshold": 0.5,
    "size": (img_width,img_width)
}


In [ ]:
def loop_through_images(content_images, style_images):
    image_set = []
    best_image_set = []
    best_cost_set = []
    image_data_logs = []
    for content_path in content_images:
        content_name = os.path.basename(content_path)
        for style_path in style_images:
            style_name = os.path.basename(style_path)
            base_image, style_reference_image, combination_image = preprocess_NST_images(
                content_path, style_path)
            generated_images, best_image, best_cost,log_data = training_loop(base_image, style_reference_image,combination_image,content_name,style_name,config=config )
            image_set.append(generated_images)
            best_image_set.append(best_image)
            best_cost_set.append(best_cost)
            image_data_logs.append(log_data)
    return image_set, best_image_set, best_cost_set, image_data_logs

c:\Users\Layo\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\models\functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(3, 400, 535, 3))
  warnings.warn(msg)


TypeError: in user code:

    File "C:\Users\Layo\AppData\Local\Temp\ipykernel_19984\189228123.py", line 7, in compute_loss_and_grads  *
        loss = compute_loss(
    File "C:\Users\Layo\AppData\Local\Temp\ipykernel_19984\2033682352.py", line 6, in compute_loss  *
        layer_features = features[content_layer_name]

    TypeError: unhashable type: 'list'


In [ ]:
image_set, best_image_set, best_cost_set, loss_logs_set, ram_usage_set, gpu_usage_set, disk_usage_set, cpu_usage_set, cpu_duration_logs_set, wall_duration_logs_set = loop_through_images(content_images, style_images)

In [ ]:
ram_usage_list = ram_usage_set[0]
best_cost_set = best_cost_set[0]
generated_images = image_set[0]
best_image = best_image_set[0]
loss_logs = loss_logs_set[0]

Get the number of iterations.

In [ ]:
number_of_iterations : list[int] = [i for i in range(len(ram_usage_list))]

In [ ]:
import matplotlib.pyplot as plt

def display_image(img):
    plt.imshow(img)
    plt.axis("off")

In [ ]:
plt.figure(figsize=(12, 12))
start_index = 0
num = len(generated_images)
for i in range(num):
    plt.subplot(4, 3, i + 1)
    display_image(generated_images[i + start_index])  # Adjust indices based on your data
plt.show()


plt.figure(figsize=(8, 8))
display_image(best_image)
plt.title("Best Image")
plt.show()

Place results into a table

In [ ]:
import pandas as pd

df = pd.DataFrame({
    "Iteration": number_of_iterations,
    "RAM Usage": [ram for ram in ram_usage_set],
    "GPU Usage ": [gpu for gpu in gpu_usage_set],
    "Disk Usage ": [disk for  disk in disk_usage_set],
    "CPU Usage": [cpu for  cpu in cpu_usage_set],
    "CPU Duration ": [cpu for  cpu in cpu_duration_logs_set],
    "Wall Duration ": [wall for wall in wall_duration_logs_set]
    })

Convert this to a table

In [ ]:
df.to_csv("usage_statistics.csv", index=False)

End the notebook at this point.

In [ ]:
import sys
sys.exit("Execution stopped here.")

# 2 Video Style Transfer 

Doing this with video.

In [ ]:
def process_frame_or_batch(base_frame_tensor, style_reference_image, img_width,img_height, optimizer):
    style_image = preprocess_image(style_reference_image, img_width, img_height)
    combination_frame_tensor = tf.Variable(base_frame_tensor)
    loss, grads = apply_style_transfer_step(combination_frame_tensor,base_frame_tensor, style_image, optimizer)

    return loss, combination_frame_tensor


In [ ]:
# Video file path
output_camera_path = "output_video.mp4"

img_height = 400
img_width = 400

Define functions for processing the video.

In [ ]:
from shared_utils.video import get_cam,load_the_video, image_read,prepare_video_writer,release_video_writer,video_end

In [ ]:
import cv2
def process_camera_frame(frame, style_image_path,img_height, img_width , optimizer):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_tensor = image_read(frame_rgb) 
    frame_tensor_resized = tf.image.resize(frame_tensor, (img_height,img_width))
    loss, processed_frame = process_frame_or_batch(frame_tensor_resized, style_image_path,img_width,img_height, optimizer)
    frame_output = deprocess_image(processed_frame.numpy(),img_width,img_height)  
    frame_color_output = cv2.cvtColor(frame_output, cv2.COLOR_RGB2BGR)
    return frame_color_output

Neural style transfer for camera.

In [ ]:
from typing import Optional, Tuple
import tensorflow as tf
import numpy as np
def apply_camera(output_path : str = "output_video.mp4",style_image_path : str = "../demo_images/starry_night.png", config = {}, video_path : str = "",verbose : int = 0):
    cam, frame_width, frame_height, fps = get_cam(video_path,video_path == "")
    lr = config.get("lr", 0.01)
    img_size = config.get("img_size", (400, 400))
    optimizer = get_optimizer(config.get("optimizer","adam"), learning_rate=lr)
    out = prepare_video_writer(output_path, frame_width, frame_height, fps)
    if not cam.isOpened() or out is None:
        print("Error: Could not open camera.")
        release_video_writer(cam,out)
        return
    title = "Camera Style Transfer" if video_path is None else "Video Style Transfer"
    start_time = time.time()
    if verbose > 0:
        print("Video path:", video_path)
        print("Output path:", output_path)
        print("Starting video processing...")
    while True:
        ret, frame = cam.read()
        if not ret:
            break  
        frame_color_output = process_camera_frame(frame, style_image_path, img_size[0], img_size[1], optimizer)
        out.write(frame_color_output)
        cv2.imshow(title, frame_color_output)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    video_end(start_time)
    release_video_writer(cam,out)
    return output_path

Do it for camera.

In [ ]:
output_camera_path = apply_camera(output_path=output_camera_path)

Do it for video.

In [ ]:
video_output_path : str = "output_video"

Prepare the configuration for the video.

In [ ]:
config = {
    "optimizer": "adam",
    "lr": 0.01,
    "img_size": (img_width, img_height),
}

In [ ]:
video_path = "../demo_video/man_at_sea_sliced.mp4"

In [ ]:
output_path = apply_camera(output_path=video_output_path,video_path=video_path, style_image_path="../demo_images/starry_night.png", config=config, verbose=1)

In [ ]:
if output_path:
    frames = load_the_video(output_path)